# 금통위 의사록 크롤링 + 전처리

## 설명 : 한국은행에서 금통위 의사록(2005-2017) pdf파일을 다운받고, pdf를 txt파일로 텍스트를 추출한다. 이 txt에서 ngram을 추출 하고난 후 콜금리 up down라벨을 붙인다

## 순서

1. 크롤링 PDF Download

2. Tika (pdf-> txt) + pdfminer

3. 의사록 전처리 - 섹션 분리, 문장 분리  @@@@@ 강조? 포인트

4. ngram추출 + 불용어 처리(정규식)  @@@@@ 강조? 포인트

5. 콜금리 라벨링

#### 사용한 자료 : 의사록 PDF 272개, 의사록 TXT 273개(1개는 직접 한글파일에서 추출), 의사록문장 3594개

### 강조 포인트

#### 3. 의사록 전처리에서

In [ ]:
parsedPDF = re.sub('\n', ' ', parser.from_file(pdf_tmp_filepath)["content"])   # \n을 space로 바꿔줌 -> 섹션구분 성능 향상

#### 3. 의사록 전처리에서

In [ ]:
sentence_enders = re.compile(r"((?<=[함음됨임봄짐움])(\s{2,3}|\.+|\()|(?<=다)\.)\s*")  # 문장구분 성능 향상

#### 4. 불용어 처리에서

In [ ]:
dr_rock['ngram'] = dr_rock['ngram'].apply(lambda x: re.sub('\s+[가-힣]{1,8}\/(JKS|JKC|JKG|JKO|JKB|JKV|JKQ|JC|JX|EP|EF|EC|ETN|ETM|XPN|XSN|XSV|XSA|XR|SF|SE|SSO|SSC|SC|SY|SH|SL|SN)(\S*|\,)','',x,50))

# 1. PDF Download 

In [2]:
import requests
from bs4 import BeautifulSoup
import urllib.request
import os

pdf_list = []


# 3~31까지
for i in range(3,30):
    url = 'http://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&pageIndex='+str(i)
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, 'html.parser')
    
    table = soup.find('div', class_='bdLine type2')
    span_1_list = table.find_all('span',class_='col m10 s10 x9 ctBx')
    span_2_list = table.find_all('div',class_='col s12 dataInfo')
    span_3_list = table.find_all('div', class_='col m2 s2 x3 fileLink')

    title_list = []
    date_list = []
    link_list = []

    for span_1 in span_1_list:
        title = span_1.find('span', class_='titlesub').text
        title_list.append(title)

    for span_2 in span_2_list:
        date = span_2.find('span', class_='date').text.replace('등록일','')
        date_list.append(date)
        
        

    for span_3 in span_3_list:
        classifi = span_3.find_all('li')[0].find('a')['title']

        if classifi[-3:] == 'hwp':
            pdf_link =span_3.find_all('li')[1].find('a')['href']
            link_list.append(pdf_link)
        else:
            pdf_link =span_3.find_all('li')[0].find('a')['href']
            link_list.append(pdf_link)

    names_urls = zip(title_list, date_list, link_list)
#         print(names_urls)

    base_pdf = 'http://www.bok.or.kr'

    for name, date, url in names_urls:
        name = date +' '+ name +'.pdf'
        pdf_list.append(name)
        url = base_pdf + url
        rq = urllib.request.Request(url)
        res = urllib.request.urlopen(rq).read()
        with open(name, mode="wb") as f:
            f.write(res)
            print("finish...")
        


finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...
finish...


# 2. PDF to txt

## 갓티카

tika

In [ ]:
!pip install tika

In [11]:
import re
from tika import parser #https://github.com/chrismattmann/tika-python
import shutil
from os import listdir
from os.path import isfile, join


# 폴더내 pdf파일을 txt 파일로 변경
# source_folder는 pdf저장된 폴더경로, output_folder는 txt저장할 폴더경로
def pdf2txt (source_folder="./data/bondreports/pdf/", output_folder="./data/bondreports/text/") :
    # 지정 폴더 내 파일 목록 조회 (파일만)
    pdf_files = [f for f in listdir(source_folder) if isfile(join(source_folder, f))]
    
    for pdf in pdf_files :
        pdf_filepath = source_folder + pdf
        pdf_tmp_filepath = output_folder + 'tmp.pdf'
        # pdf 파일을 text로 변환

        
        shutil.copyfile(pdf_filepath, pdf_tmp_filepath)
        
        # 실제 parsing하는 코드는 이 1줄
        parsedPDF = re.sub('\n', ' ', parser.from_file(pdf_tmp_filepath)["content"])
#         parsedPDF = parsedPDF.replace('  ','')
        #print(parsedPDF)

        ouput_filepath = (output_folder + pdf).replace('.pdf', '.txt')
        with open(ouput_filepath, 'w', encoding='utf8' ) as f :
            print(ouput_filepath)
            f.write(parsedPDF)
            f.close()


In [ ]:
# 경로 마지막에 /를 붙이는 것이 포인트 -킹영창曰

pdf2txt('/Users/seonmin/NLP_Hause/자연어처리/논문 자료수집/금통위 의사록 pdf/', '/Users/seonmin/NLP_Hause/자연어처리/논문 자료수집/금통위 의사록 텍스트/')

### 개뚜래기 pdfminer

In [ ]:
pip install pdfminer.six

In [ ]:
https://github.com/pdfminer/pdfminer.six/issues/210
https://github.com/pdfminer/pdfminer.six/pull/228
    
참고해서 본인 pdffond.py 파일 수정하고 진행


In [ ]:
import sys
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import XMLConverter, HTMLConverter, TextConverter
from pdfminer.layout import LAParams
import io
import os


def pdfparser(data):

    fp = open(data, 'rb')
    rsrcmgr = PDFResourceManager()
    retstr = io.StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    # Create a PDF interpreter object.
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    # Process each page contained in the document.

    for page in PDFPage.get_pages(fp):
        interpreter.process_page(page)
        data =  retstr.getvalue()

#     print(data)
    return data


if __name__ == '__main__':
#     pdfparser(sys.argv[1])
#     for i in pdf_list:
    for i in pdf_list:
        try:
            with open(i + '.txt' , 'w', encoding='utf-8') as f:
                a = pdfparser(i)
                f.write(a)
            oldname = i + '.txt'
            newname = oldname[:-8]+'.txt'
            os.rename(oldname, newname )
        except:
            print('오류: ',i)
            

## 예외항목 자료 가져오기

#### 2005년 제10차 금통위 의사록 hwp파일. 1개만 직접 가져와서 TXT로 복사붙여넣기해서 만든다

In [ ]:
# http://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&pageIndex=30
    
# 1. 위 사이트에서 의사록 목록 중 2005.05 이후부터의 파일 3개를 직접 다운받는다.
# 게시글: 2005년도 제13차, 2005년도 제 12차, 2005년도 제 10차
#     * 2005년도 제 10차는 hwp파일만 있으므로 주의한다.

# 2. 파일을 다운받으면 다음과 같은 이름으로 저장된다. 
# 2005년 제10차 금통위 의사록 # hwp파일
# 2005년 제13차 금통위 의사록 - PDF파일
# 2005년 제12차 금통위 의사록 - PDF파일

# # 3. 위에서 활용한 pdfminer로 2개 pdf 파일을 txt파일로 변환한다.

# 4. 나머지 hwp파일은 직접 한글파일을 열어서 ctrl+a과 ctrl+c,v 활용하여 txt파일을 만든다

## 피클에 pdf list 담아놓기 

#### 이 주피터노트북을 다시 쓸 때 다시 PDF를 크롤링하지 않기 위해 만드는 PDF LIST

In [4]:
import pickle

with open('pdf_list.txt', 'wb') as f:
    pickle.dump(pdf_list, f)

In [5]:
# 원래 저장했던 리스트 타입으로 반환함

with open('pdf_list.txt', 'rb') as f:
    data = pickle.load(f)
    print(data)

['2018.06.12 금융통화위원회 의사록(2018년도 제10차)(2018.5.24.) .pdf', '2018.05.15 금융통화위원회 의사록(2018년도 제8차)(2018.4.26.).pdf', '2018.05.02 금융통화위원회 의사록(2018년도 제7차)(2018.4.12.).pdf', '2018.04.17 금융통화위원회 의사록(2018년도 제6차)(2018.3.29.).pdf', '2018.03.27 금융통화위원회 의사록(2018년도 제5차)(2018.3.8.).pdf', '2018.03.20 금융통화위원회 의사록(2018년도 제4차)(2018.2.27.).pdf', '2018.02.27 금융통화위원회 의사록(2018년도 제3차)(2018.2.8.).pdf', '2018.02.06 금융통화위원회 의사록(2018년도 제1차)(2018.1.18.).pdf', '2018.01.16 금융통화위원회 의사록(2017년도 제25차)(2017.12.28.).pdf', '2018.01.02 금융통화위원회 의사록(2017년도 제24차)(2017.12.14.).pdf', '2017.12.26 금융통화위원회 의사록(2017년도 제23차)(2017.12.7.).pdf', '2017.12.19 금융통화위원회 의사록(2017년도 제22차)(2017.11.30.).pdf', '2017.11.28 금융통화위원회 의사록(2017년도 제21차)(2017.11.9.).pdf', '2017.11.07 금융통화위원회 의사록(2017년도 제19차)(2017.10.19.).pdf', '2017.10.10 금융통화위원회 의사록(2017년도 제18차)(2017.9.21.).pdf', '2017.09.19 금융통화위원회 의사록(2017년도 제16차)(2017.8.31.).pdf', '2017.08.29 금융통화위원회 의사록(2017년도 제15차)(2017.8.10.).pdf', '2017.08.01 금융통화위원회 의사록(2017년도 제13차)(2017.7.13.).pdf', '2017.07.07 금

In [19]:
# 2005년 5월 첫 의사록은 한글파일이므로 직접 txt로 복붙하여 만든다
# tika를 거친게 아니므로, 목록에 없다. 따라서 여기서 직접 추가해준다
txt_list = []
for i in data:
    txt_list.append(i.replace('.pdf',''))

extra_txt = '2005.06.28 금융통화위원회 의사록(2005년도 제10차)(2005.5.12)'
txt_list.append(extra_txt)

## 금통위 의사록 전처리

# 3. 의사록 섹션 분리

#### 우리에게 주어진 코드로는 의사록 섹션과 문장이 명확히 구분되지 않는 문제 발생

#### 코드 내에 있는 정규식과 tika코드를 약간 수정하여 원하는 섹션과 문장을 구분하도록 만듦

이 부분 수정함

re.compile(r"((?<=[함음됨임봄짐움])(\s{2,3}|\.+|\()|(?<=다)\.)\s*")

In [12]:
def tidy_sentences(section):
#     sentence_enders = re.compile(r"((?<=[함음됨임봄짐움])(\s*\n|\.|;)|(?<=다)\.)\s*") <- 원본
    sentence_enders = re.compile(r"((?<=[함음됨임봄짐움])(\s{2,3}|\.+|\()|(?<=다)\.)\s*")  #######
    
    splits = list((m.start(), m.end()) for m in re.finditer(sentence_enders, section))
    starts = [0] + [i[1] for i in splits]
    ends = [i[0] for i in splits]
    sentences = [section[start:end] for start, end in zip(starts[:-1], ends)]
    for i, s in enumerate(sentences):
        sentences[i] = (s.replace('\n', ' ').replace('  ', '')) + '.' ### 여기를 건드림
    #쓰지도 않는 부분
    text = '\n'.join(sentences) if len(sentences) > 0 else ''
    
    return sentences, text

import re 

def preprocess_minutes(minutes):

    pos = re.search('(.?국내외\s?경제\s?동향.?과 관련하여,?|\(가\).+경제전망.*|\(가\) 국내외 경제동향 및 평가)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s1 = pos.start() if pos else -1
    pos = re.search('(.?외환.?국제금융\s?동향.?과 관련하여.*|\(나\) 외환.국제금융\s?(및 금융시장)?\s?동향)\n?\s*(일부 위원은|대부분의 위원들은)', minutes,re.MULTILINE)
    s2 = pos.start() if pos else -1
    pos = re.search('(.?금융시장\s?동향.?과 관련하여,?|\(다\) 금융시장\s?동향)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s3 = pos.start() if pos else -1
    pos = re.search('((\((다|라)\) )?.?통화정책\s?방향.?에 관한 토론,?|이상과 같은 의견\s?교환을 바탕으로.*통화정책\s?방향.*에.*토론.*)\n?', minutes,re.MULTILINE)
    s4 = pos.start() if pos else -1
    pos = re.search('(\(4\) 정부측 열석자 발언.*)\n?', minutes, re.MULTILINE)
    s5 = pos.start() if pos else -1
    pos = re.search('(\(.*\) 한국은행 기준금리 결정에 관한 위원별 의견\s?개진|이상과 같은 토론에 이어 .* 관한 위원별 의견개진이 있었음.*)\n?', minutes,re.MULTILINE)
    s6 = pos.start() if pos else -1
    positer = re.finditer('(\(\s?.*\s?\) ()(심의결과|토의결론))\n?', minutes, re.MULTILINE)
    s7 = [pos.start() for pos in positer if pos.start() > s6]
    s7 = s7[0] if s7 else -1

    # 국내외 경제동향
    bos = s1
    eos = s2
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section1, section1_txt = tidy_sentences(section)

    # 외환․국제금융 동향
    bos = s2
    eos = s3 if s3 >= 0 else s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section2, section2_txt = tidy_sentences(section)
    #print(section)

    # 금융시장 동향
    bos = s3
    eos = s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section3, section3_txt = tidy_sentences(section)

    # 통화정책방향
    bos = s4
    eos = s5 if s5 >= 0 else s6 if s6 >= 0 else s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section4, section4_txt = tidy_sentences(section)

    # 위원별 의견 개진
    bos = s6
    eos = s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section5, section5_txt = tidy_sentences(section)

    # 정부측 열석자 발언
    bos = s5
    eos = s6
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('정부측 열석자 발언', section, re.MULTILINE)
    bos = pos.end() + 1 if pos else -1
    section = section[bos:] if bos >= 0 else section
    section6, section6_txt = tidy_sentences(section)

    sections = ['Economic Situation', 'Foreign Currency', 'Financial Markets',
                'Monetary Policy', 'Participants’ Views', 'Government’s View']
    section_texts = (section2, section3)


    return section_texts

## 의사록 섹션구분

In [58]:
import pandas as pd

#### 경로를 txt파일이 있는 곳으로 지정한다

In [188]:
cd C:\Users\seonmin\NLP_Hause\자연어처리\논문 자료수집\금통위 의사록 텍스트 - 복사본

C:\Users\seonmin\NLP_Hause\자연어처리\논문 자료수집\금통위 의사록 텍스트 - 복사본


In [187]:
txt_list[0][-11:].replace(')','').strip()

'2018.5.24.'

#### txt파일 앞에 날짜가 있으므로, txt제목에서 날짜만 추출해서 list에 담는다

In [202]:

date_list = []
for i in txt_list:
#     date_list.append(i[:10])
    date_list.append(i[-12:].replace(')','').replace('(','').replace('차','').strip())

    
# txt에서 추출한 텍스트로 섹션을 구분한다
# dict 자료형으로 저장하는데, 여기서 key값은 위에서 담아놓았던 date_list의 값을 활용한다.
a_dict = {}  
for idx,val in enumerate(txt_list):
    with open(val+'.txt','r',encoding='utf-8' ) as f:
        try:
            text = f.read() 
            a = preprocess_minutes(text)
            a_dict[date_list[idx]] = a
            
        except:
            print(val)
        

#### 전체 txt파일 중 몇 개의 파일을 섹션구분했는지 확인한다

In [203]:

len(a_dict)

271

In [204]:
a_dict['2006.10.24.']

KeyError: '2006.10.24.'

In [ ]:
len(a_dict['2018.06.12'][0])

In [205]:
date_list

['2018.5.24.',
 '2018.4.26.',
 '2018.4.12.',
 '2018.3.29.',
 '2018.3.8.',
 '2018.2.27.',
 '2018.2.8.',
 '2018.1.18.',
 '2017.12.28.',
 '2017.12.14.',
 '2017.12.7.',
 '2017.11.30.',
 '2017.11.9.',
 '2017.10.19.',
 '2017.9.21.',
 '2017.8.31.',
 '2017.8.10.',
 '2017.7.13.',
 '2017.6.22.',
 '2017.5.25.',
 '2017.4.27.',
 '2017.4.13.',
 '2017.3.23.',
 '2017.2.23.',
 '2017.1.26.',
 '2017.1.13.',
 '2016.12.29.',
 '2016.12.21.',
 '2016.12.15.',
 '2016.11.24.',
 '2016.11.11.',
 '2016.10.27.',
 '2016.10.13.',
 '2016.9.22.',
 '2016.9.9.',
 '2016.8.11.',
 '2016.7.14.',
 '2016.7.1.',
 '2016.6.23.',
 '2016.6.9.',
 '2016.5.13.',
 '2016.4.19.',
 '2016.3.24.',
 '2016.3.10.',
 '2016.2.25.',
 '2016.2.16',
 '2016.1.28.',
 '2016.1.14.',
 '2015.12.24.',
 '2015.12.16.',
 '2015.12.10.',
 '2015.11.26.',
 '2015.11.12.',
 '2015.10.29.',
 '2015.10.15.',
 '2015.9.24.',
 '2015.9.11.',
 '2015.8.27.',
 '2015.8.13.',
 '2015.7.23.',
 '2015.7.9.',
 '2015.6.25.',
 '2015.6.11.',
 '2015.5.15.',
 '2015.4.23.',
 '2015.4.9.',


## 의사록을 dataframe형태로 만들기

### 최종적으로 만들어야하는 df의 컬럼명들

### 날짜, 섹션2, 섹션3, ngram 

In [206]:
# 빈 df를 만들고 
# 의사록 파일 1개당 1개의 df를 만들어서 append 시킨다
# .T로 전치시켜서 붙인다
import pandas as pd
# total_df = pd.DataFrame(columns=['빈'])
s2_df = pd.DataFrame(columns = ['date','category','content'])
s3_df = pd.DataFrame(columns = ['date','category','content'])
total_df=pd.DataFrame(columns = ['date','content'])


for i in date_list:
    for j in a_dict[i][0]:
        a_date = i
        a_s2 = j
        temp_df = pd.DataFrame([[a_date,'S2',a_s2]],columns=['date','category','content'])
        s2_df = s2_df.append(temp_df, ignore_index = True)    

for i in date_list:
    for j in a_dict[i][1]:
        a_date = i
        a_s3 = j
        temp_df = pd.DataFrame([[a_date,'S3',a_s3]],columns=['date','category','content'])
        s3_df = s3_df.append(temp_df, ignore_index = True) 
        
total_df = s2_df.append(s3_df,ignore_index = True)

### df의 컬럼을 date, category, content로 단순화

#### Category의 S2는 섹션2, S3는 섹션3

In [207]:
total_df

,date,category,content
0,2018.5.24.,S2,일부 위원은 최근의 미 달러화 강세가 추가적으로 진행될 경우 일부 취약국에 국한된 ...
1,2018.5.24.,S2,이어서 동 위원은 최근 회사채 시장동향과 관련하여 일부에서는 경기개선 및금리 메리트...
2,2018.5.24.,S2,"이에 대해 관련부서에서는 지난해 경기회복에 따른 기업실적 개선, 금융기관의 신용경계..."
3,2018.5.24.,S2,이에 동 위원은 지난 몇 년간 부채감축(deleveraging)을 진행해 온 건설회...
4,2018.5.24.,S2,이에 대해 관련부서에서는 그간 건설업종이 취약업종으로 분류되어 투자대상으로 기피되었...
5,2018.5.24.,S2,또한 동 위원은 우리나라 금융시장에서 위험선호(risk-on) 현상이 두드러지게나타...
6,2018.5.24.,S2,이에 대해 관련부서에서는 지정학적 리스크 완화도 일부 영향을 미친 것으로보이지만 회...
7,2018.5.24.,S2,아울러 동 위원은 최근 전세대출의 큰 폭 증가는 아파트 신규분양 이후 잔금납부의 어...
8,2018.5.24.,S2,이에 관련부서에서는 아파트 신규입주 증가로 전세거래가 증가한 점이 전세자금대출을 증...
9,2018.5.24.,S2,다음으로 동 위원은 대기업대출 증가가 경기개선에 따른 운전자금수요 확대등에 기인한 ...


#### 불필요한 문자를 제거하는 작업

In [208]:
# 


total_df = total_df.astype('str')

# '지우기
total_df['content'] = total_df['content'].apply(lambda x: re.sub("'",'',x,20))
# , 지우기
total_df['content'] = total_df['content'].apply(lambda x: re.sub(",",'',x,100))
# [] 지우기
total_df['content'] = total_df['content'].apply(lambda x: re.sub("\[",'',x,10))
total_df['content'] = total_df['content'].apply(lambda x: re.sub("\]",'',x,10))
total_df['content'] = total_df['content'].apply(lambda x: re.sub("\-",'',x,10))




In [ ]:
여기까지 ㅇㅋ

# 4. ngram 추출하기

#### n-gram 코드

In [211]:
# 

import pandas as pd
from ekonlpy.sentiment import MPCK


# return을 ngram과 token 각각 받는다
def text2ngram(text):
    mpck = MPCK()
    bef_tokens = mpck.tokenize(text)
    ngrams = mpck.ngramize(bef_tokens)
    return ngrams, bef_tokens 

#### ngram과 token을 각각 리스트에 임시 저장한다
시간이 아주 많이 걸리므로 돌려놓고 잠깐 쉬도록 하자

In [212]:


tokens = []
ngrams = []

for i in range(len(total_df)):
    a, b = text2ngram(total_df['content'][i])
    ngrams.append(a)
    tokens.append(b)
 


In [213]:
# ngram과 token을 하나의 리스트로 합친다.
# 시간이 아주 많이 걸리므로 돌려놓고 잠깐 쉬도록 하자

ngram_total =[]
for i in range(len(ngrams)):
    ngram_total.append(ngrams[i]+tokens[i])
    
len(ngram_total)



# s3_total =[]
# for i in range(len(s3_ngrams)):
#     s3_total.append(s3_ngrams[i]+s3_tokens[i])
    
# len(s3_total)





3594

In [214]:
ngram_df = pd.DataFrame({'ngram':ngram_total})

In [215]:
dr_rock = pd.merge(total_df, ngram_df, left_index = True, right_index=True)


#### ngram이 담긴 df를 확인한다

In [216]:
dr_rock

,date,category,content,ngram
0,2018.5.24.,S2,일부 위원은 최근의 미 달러화 강세가 추가적으로 진행될 경우 일부 취약국에 국한된 ...,"[미/NNG, 달러/NNG, 강세/NNG, 추가/NNG, 진행/NNG, 되/VV, ..."
1,2018.5.24.,S2,이어서 동 위원은 최근 회사채 시장동향과 관련하여 일부에서는 경기개선 및금리 메리트...,"[호조/NNG;경기/NNG;회복/NNG, 경기/NNG;개선/NNG, 채권/NNG;위..."
2,2018.5.24.,S2,이에 대해 관련부서에서는 지난해 경기회복에 따른 기업실적 개선 금융기관의 신용경계감...,"[가계/NNG;대출/NNG;관리/NNG;강화/NNG, 기업/NNG;실적/NNG;개선..."
3,2018.5.24.,S2,이에 동 위원은 지난 몇 년간 부채감축(deleveraging)을 진행해 온 건설회...,"[부채/NNG;감축/NNG, 부채/NNG, 감축/NNG, 진행/NNG, 건설/NNG..."
4,2018.5.24.,S2,이에 대해 관련부서에서는 그간 건설업종이 취약업종으로 분류되어 투자대상으로 기피되었...,"[대해/VV, 취약/NNG, 업종/NNG, 분류/NNG, 되/XSV, 투자대상/NN..."
5,2018.5.24.,S2,또한 동 위원은 우리나라 금융시장에서 위험선호(riskon) 현상이 두드러지게나타나...,"[금융시장/NNG;위험/NNG;선호/NNG, 지정학/NNG;위험/NNG;완화/NNG..."
6,2018.5.24.,S2,이에 대해 관련부서에서는 지정학적 리스크 완화도 일부 영향을 미친 것으로보이지만 회...,"[지정학/NNG;위험/NNG;완화/NNG, 대해/VV, 지정학/NNG, 위험/NNG..."
7,2018.5.24.,S2,아울러 동 위원은 최근 전세대출의 큰 폭 증가는 아파트 신규분양 이후 잔금납부의 어...,"[아울러/MAG, 전세/NNG, 대출/NNG, 크/VA, 증가/NNG, 신규/NNG..."
8,2018.5.24.,S2,이에 관련부서에서는 아파트 신규입주 증가로 전세거래가 증가한 점이 전세자금대출을 증...,"[전세자금/NNG;대출/NNG;증가/NNG, 신규/NNG, 입주/NNG, 증가/NN..."
9,2018.5.24.,S2,다음으로 동 위원은 대기업대출 증가가 경기개선에 따른 운전자금수요 확대등에 기인한 ...,"[대출/NNG;증가/NNG, 경기/NNG;개선/NNG, 수요/NNG;확대/NNG, ..."


#### ngram 정제작업을 거친다. 
작은 따옴표는 2000정도로 충분히 제거해야한다.


In [217]:

import re

dr_rock = dr_rock.astype('str')

dr_rock['ngram'] = dr_rock['ngram'].apply(lambda x: re.sub("'",'',x,2000))
dr_rock['ngram'] = dr_rock['ngram'].apply(lambda x: re.sub("\[",'',x,10))
dr_rock['ngram'] = dr_rock['ngram'].apply(lambda x: re.sub("\]",'',x,10))

In [218]:
# ngram정제가 잘 되었는지 확인한다
dr_rock['ngram']

0       미/NNG, 달러/NNG, 강세/NNG, 추가/NNG, 진행/NNG, 되/VV, 취...
1       호조/NNG;경기/NNG;회복/NNG, 경기/NNG;개선/NNG, 채권/NNG;위험...
2       가계/NNG;대출/NNG;관리/NNG;강화/NNG, 기업/NNG;실적/NNG;개선/...
3       부채/NNG;감축/NNG, 부채/NNG, 감축/NNG, 진행/NNG, 건설/NNG,...
4       대해/VV, 취약/NNG, 업종/NNG, 분류/NNG, 되/XSV, 투자대상/NNG...
5       금융시장/NNG;위험/NNG;선호/NNG, 지정학/NNG;위험/NNG;완화/NNG,...
6       지정학/NNG;위험/NNG;완화/NNG, 대해/VV, 지정학/NNG, 위험/NNG,...
7       아울러/MAG, 전세/NNG, 대출/NNG, 크/VA, 증가/NNG, 신규/NNG,...
8       전세자금/NNG;대출/NNG;증가/NNG, 신규/NNG, 입주/NNG, 증가/NNG...
9       대출/NNG;증가/NNG, 경기/NNG;개선/NNG, 수요/NNG;확대/NNG, 대...
10      경기/NNG;개선/NNG, 매출/NNG;증가/NNG, 대해/VV, 통상/NNG, 단...
11      기업/NNG, 중/NNG, 기업/NNG, 창출/NNG, 하/XSV, 수익/NNG, ...
12      기업/NNG, 실적/NNG, 영업이익/NNG, 기준/NNG, 수익률/NNG, 높/V...
13      실적/NNG;악화/NNG, 실적/NNG, 대비/NNG, 감소/NNG, 하/XSV, ...
14      유동성/NNG, 자금/NNG, 풍부/NNG, 가운데/NNG, 가계/NNG, 과잉/N...
15      미/NNG;장기/NNG;금리/NNG;상승/NNG, 금리/NNG;경기/NNG;개선/N...
16      미/NNG;장기/NNG;금리/NNG;상승/NNG, fed/NNG;금리/NNG;인상/...
17      유동성/NN

In [219]:
# 날짜를 datetime으로 변환해준다
dr_rock['date'] = pd.to_datetime(dr_rock['date'])

## 불용어 처리

### tokens에서 우리가 사용하지 않는 품사를 지워주는 작업

#### tokens에서 필요한 품사를 제외한 나머지 품사를 지워준다. 

In [ ]:


dr_rock['ngram'] = dr_rock['ngram'].apply(lambda x: re.sub('\s+[가-힣]{1,8}\/(JKS|JKC|JKG|JKO|JKB|JKV|JKQ|JC|JX|EP|EF|EC|ETN|ETM|XPN|XSN|XSV|XSA|XR|SF|SE|SSO|SSC|SC|SY|SH|SL|SN)(\S*|\,)','',x,50))

In [220]:
# 전체 데이터를 확인한다
dr_rock

,date,category,content,ngram
0,2018-05-24,S2,일부 위원은 최근의 미 달러화 강세가 추가적으로 진행될 경우 일부 취약국에 국한된 ...,"미/NNG, 달러/NNG, 강세/NNG, 추가/NNG, 진행/NNG, 되/VV, 취..."
1,2018-05-24,S2,이어서 동 위원은 최근 회사채 시장동향과 관련하여 일부에서는 경기개선 및금리 메리트...,"호조/NNG;경기/NNG;회복/NNG, 경기/NNG;개선/NNG, 채권/NNG;위험..."
2,2018-05-24,S2,이에 대해 관련부서에서는 지난해 경기회복에 따른 기업실적 개선 금융기관의 신용경계감...,"가계/NNG;대출/NNG;관리/NNG;강화/NNG, 기업/NNG;실적/NNG;개선/..."
3,2018-05-24,S2,이에 동 위원은 지난 몇 년간 부채감축(deleveraging)을 진행해 온 건설회...,"부채/NNG;감축/NNG, 부채/NNG, 감축/NNG, 진행/NNG, 건설/NNG,..."
4,2018-05-24,S2,이에 대해 관련부서에서는 그간 건설업종이 취약업종으로 분류되어 투자대상으로 기피되었...,"대해/VV, 취약/NNG, 업종/NNG, 분류/NNG, 되/XSV, 투자대상/NNG..."
5,2018-05-24,S2,또한 동 위원은 우리나라 금융시장에서 위험선호(riskon) 현상이 두드러지게나타나...,"금융시장/NNG;위험/NNG;선호/NNG, 지정학/NNG;위험/NNG;완화/NNG,..."
6,2018-05-24,S2,이에 대해 관련부서에서는 지정학적 리스크 완화도 일부 영향을 미친 것으로보이지만 회...,"지정학/NNG;위험/NNG;완화/NNG, 대해/VV, 지정학/NNG, 위험/NNG,..."
7,2018-05-24,S2,아울러 동 위원은 최근 전세대출의 큰 폭 증가는 아파트 신규분양 이후 잔금납부의 어...,"아울러/MAG, 전세/NNG, 대출/NNG, 크/VA, 증가/NNG, 신규/NNG,..."
8,2018-05-24,S2,이에 관련부서에서는 아파트 신규입주 증가로 전세거래가 증가한 점이 전세자금대출을 증...,"전세자금/NNG;대출/NNG;증가/NNG, 신규/NNG, 입주/NNG, 증가/NNG..."
9,2018-05-24,S2,다음으로 동 위원은 대기업대출 증가가 경기개선에 따른 운전자금수요 확대등에 기인한 ...,"대출/NNG;증가/NNG, 경기/NNG;개선/NNG, 수요/NNG;확대/NNG, 대..."


# 5. 콜금리 라벨링 작업

#### 경로 설정

In [221]:
cd C:\Users\seonmin\NLP_Hause\자연어처리\논문 자료수집

C:\Users\seonmin\NLP_Hause\자연어처리\논문 자료수집


#### 미리 만들어둔 콜금리 csv 불러오기

In [222]:
my_callrate = pd.read_csv('my_callrate_ffill.csv', encoding='utf-8',index_col=0)

In [223]:
# index_col=0 안 할 경우
my_callrate = my_callrate.drop('Unnamed: 0', axis = 1)

In [224]:
my_callrate

,date,old_date,callrate,old_callrate,updown
0,2019-07-15,2019-06-15,1.73,1.74,down
1,2019-07-14,2019-06-15,1.73,1.74,down
2,2019-07-13,2019-06-15,1.73,1.74,down
3,2019-07-12,2019-06-12,1.74,1.75,down
4,2019-07-11,2019-06-11,1.75,1.75,neutrality
5,2019-07-10,2019-06-10,1.74,1.75,down
6,2019-07-09,2019-06-09,1.69,1.75,down
7,2019-07-08,2019-06-08,1.73,1.77,down
8,2019-07-07,2019-06-08,1.73,1.77,down
9,2019-07-06,2019-06-08,1.73,1.77,down


#### 날짜 datetime으로 바꿔서 병합할 준비

In [225]:
my_callrate['date'] = pd.to_datetime(my_callrate['date'])

#### 왼쪽에 의사록을 놓고, date를 기준으로 병합시도.

#### 왼쪽의 데이터는 100% 살려두고 기준on과 맞는 오른쪽 데이터만 가져와서 왼쪽에 붙이는 left조인을 한다

In [226]:

dr_rock_last = pd.merge(dr_rock, my_callrate, on='date', how='left')

In [227]:
dr_rock_last = dr_rock_last.drop('old_date',axis = 1).drop('callrate', axis = 1).drop('old_callrate', axis = 1)

In [228]:
dr_rock_last

,date,category,content,ngram,updown
0,2018-05-24,S2,일부 위원은 최근의 미 달러화 강세가 추가적으로 진행될 경우 일부 취약국에 국한된 ...,"미/NNG, 달러/NNG, 강세/NNG, 추가/NNG, 진행/NNG, 되/VV, 취...",up
1,2018-05-24,S2,이어서 동 위원은 최근 회사채 시장동향과 관련하여 일부에서는 경기개선 및금리 메리트...,"호조/NNG;경기/NNG;회복/NNG, 경기/NNG;개선/NNG, 채권/NNG;위험...",up
2,2018-05-24,S2,이에 대해 관련부서에서는 지난해 경기회복에 따른 기업실적 개선 금융기관의 신용경계감...,"가계/NNG;대출/NNG;관리/NNG;강화/NNG, 기업/NNG;실적/NNG;개선/...",up
3,2018-05-24,S2,이에 동 위원은 지난 몇 년간 부채감축(deleveraging)을 진행해 온 건설회...,"부채/NNG;감축/NNG, 부채/NNG, 감축/NNG, 진행/NNG, 건설/NNG,...",up
4,2018-05-24,S2,이에 대해 관련부서에서는 그간 건설업종이 취약업종으로 분류되어 투자대상으로 기피되었...,"대해/VV, 취약/NNG, 업종/NNG, 분류/NNG, 되/XSV, 투자대상/NNG...",up
5,2018-05-24,S2,또한 동 위원은 우리나라 금융시장에서 위험선호(riskon) 현상이 두드러지게나타나...,"금융시장/NNG;위험/NNG;선호/NNG, 지정학/NNG;위험/NNG;완화/NNG,...",up
6,2018-05-24,S2,이에 대해 관련부서에서는 지정학적 리스크 완화도 일부 영향을 미친 것으로보이지만 회...,"지정학/NNG;위험/NNG;완화/NNG, 대해/VV, 지정학/NNG, 위험/NNG,...",up
7,2018-05-24,S2,아울러 동 위원은 최근 전세대출의 큰 폭 증가는 아파트 신규분양 이후 잔금납부의 어...,"아울러/MAG, 전세/NNG, 대출/NNG, 크/VA, 증가/NNG, 신규/NNG,...",up
8,2018-05-24,S2,이에 관련부서에서는 아파트 신규입주 증가로 전세거래가 증가한 점이 전세자금대출을 증...,"전세자금/NNG;대출/NNG;증가/NNG, 신규/NNG, 입주/NNG, 증가/NNG...",up
9,2018-05-24,S2,다음으로 동 위원은 대기업대출 증가가 경기개선에 따른 운전자금수요 확대등에 기인한 ...,"대출/NNG;증가/NNG, 경기/NNG;개선/NNG, 수요/NNG;확대/NNG, 대...",up


#### 이상이 없다면, csv로 저장한다. 

In [229]:
# 

dr_rock_last.to_csv('(완성)의사록 ngram + label_수정.csv', encoding='UTF-8' )

#### 확인하기

In [230]:
dr_rock_last[183:]

,date,category,content,ngram,updown
183,2017-11-30,S2,일부 위원은 최근 미국의 금융·경제상황을 보면 금년 12월 미 연준이 금리를인상할 ...,"fed/NNG;금리/NNG;인상/NNG, 견조/NNG;회복/NNG, 금융경제/NNG...",up
184,2017-11-30,S2,일각에서는 미국의 장기실질균형금리 및 기대인플레이션의 하락을 그 원인으로 거론하고 ...,"기대/NNG;인플레이션/NNG;하락/NNG, 일각/NNG, 장기/NNG, 실질/NN...",up
185,2017-11-30,S2,한편 이러한 현상을 향후 실물경기의 침체 우려와 글로벌 장기채권 수요의 유입에 따른...,"실물/NNG;경기/NNG;침체/NNG;우려/NNG, 이러/NNG, 현상/NNG, 실...",up
186,2017-11-30,S2,이에 대해 관련부서에서는 수익률곡선의 평탄화나 장단기금리차의 역전이 장기금리의 하락...,"단기물/NNG;발행/NNG;확대/NNG, fed/NNG;금리/NNG;인상/NNG, ...",up
187,2017-11-30,S2,이에 동 위원은 미국의 세제개편안이 의회를 통과한다면 수익률곡선이나 정책금리 경로가...,"세제개편안/NNG, 통과/NNG, 하/VV, 일드커브/NNG, 금리/NNG, 경로/...",up
188,2017-11-30,S2,비록 일부 선반영된 측면도 있겠지만미 의회에서 세제개편안을 가결한다면 재정적자 보전...,"재정/NNG;적자/NNG, 비록/MAG, 선반영/NNG, 되/VV, 측면/NNG, ...",up
189,2017-11-30,S2,이와 관련하여 동 위원은 한·미 간 장기금리의 동조화 경향에도 불구하고 우리나라의 ...,"금리/NNG;인상/NNG, 미/NNG, 장기/NNG, 금리/NNG, 동조/NNG, ...",up
190,2017-11-30,S2,이어서 동 위원은 원/달러 환율과의 마이너스 상관관계가 높아지면서 거주자외화예금이 ...,"변동성/NNG;완화/NNG, 지표/NNG;플러스/NNG, 이어서/MAG, 원달러/N...",up
191,2017-11-30,S2,이에 동 위원은 이들 지표 간의 마이너스 상관관계가 강화된 것은 외환시장의 복원력 ...,"지표/NNG;마이너스/NNG, 지표/NNG, 마이너스/NNG, 상관관계/NNG, 강...",up
192,2017-11-30,S2,또한 동 위원은 이번 가계부채 종합대책을 보면 주로 다주택자와 분양시장에초점을 두고...,"가계/NNG, 부채/NNG, 종합/NNG, 대책/NNG, 보/VV, 주로/MAG, ...",up
